In [6]:
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity

file_path = "./data/test.csv"
df = pd.read_csv(file_path, header=0)



In [7]:


#return an array of all jobs rating (existing + predicted) for the given user
def job_job_collab_filter(uid, df, k=3):
    job_corr = df.T.corr(method='pearson', numeric_only=True)
    global_mean = sum(df.sum(skipna=True))/df.size
    uid_ratings = df.iloc[:,uid]
    uid_baseline = uid_ratings.mean(skipna=True) - global_mean
    df_row_diff = df.sub(df.mean(axis=1), axis=0)
    cos_sim = cosine_similarity(df_row_diff.fillna(0))
    # print(f'uid base is: ', uid_baseline)
    for jid, rating in enumerate(uid_ratings):
        if math.isnan(rating): #no existing user rating, use cf
            # print(f'jid is: ', jid)
            temp_ratings_uid = uid_ratings.fillna(0)
            # prep k largest for cos_sim
            largest_k_indicies = np.argpartition(cos_sim[:,jid],-k)[-k:]
            k_nearest_sims = cos_sim[:,jid]
            for i in range(len(cos_sim[:,jid])):
                if i not in largest_k_indicies:
                    k_nearest_sims[i] = 0
            # print(f'k_nearest sims: ', k_nearest_sims)
            # print(f'dot prod with cos_sim: ', temp_ratings_uid.dot(k_nearest_sims))
            # print(f'sum of sims: ', sum(cos_sim[:,jid]))
            # print(f'normalizeed dot prod with cos_sim is: ', temp_ratings_uid.dot(k_nearest_sims)/sum(k_nearest_sims))
            # print(f'predicted rating is: ', temp_ratings_uid.dot(k_nearest_sims)/sum(k_nearest_sims) + cos_sim[:,jid].mean() + uid_baseline)

            uid_ratings[jid] = temp_ratings_uid.dot(k_nearest_sims)/sum(k_nearest_sims)\
                + uid_baseline + cos_sim[:,jid].mean()
        else:
            pass
    return uid_ratings

uid = 2
print(f'ratings for uid', uid, ' is: \n', job_job_collab_filter(uid,df))

ratings for uid 2  is: 
 0    2.941754
1    3.617122
2    3.000000
3    2.639038
4    3.673149
5    4.000000
6    3.000000
Name: 2, dtype: float64
